In [16]:
!pip install pyvis
!pip install networkx

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns

from pyvis.network import Network 
import networkx as nx

In [33]:
df = pd.read_csv("origin_destination_train_202302.csv")
df1 = df.loc[df.DAY_TYPE == "WEEKDAY"]
df2 = df.loc[df.DAY_TYPE == "WEEKENDS/HOLIDAY"]

df1 = df

In [34]:
df3 = pd.read_csv("weekday_passenger_vol.csv").rename(columns={"PT_CODE":"ORIGIN_PT_CODE"})
df4 = pd.read_csv("weekends_passenger_vol.csv").rename(columns={"PT_CODE":"ORIGIN_PT_CODE"})

In [35]:
df3 = df3.append(df4)

C:\Users\leish\AppData\Local\Temp\ipykernel_14136\3968143010.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(df4)


In [36]:
df1 = df1.merge(df3[["ORIGIN_PT_CODE","TIME_PER_HOUR", "DAY_TYPE",'TOTAL_TAP_IN_VOLUME']], on=["ORIGIN_PT_CODE","TIME_PER_HOUR", "DAY_TYPE"])

In [37]:
df1 = df1.drop(df1.loc[(df1.ORIGIN_PT_CODE.str.startswith("B")) | (df1.ORIGIN_PT_CODE.str.startswith("P"))| (df1.ORIGIN_PT_CODE.str.startswith("S"))].index)

In [38]:
df1

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,TOTAL_TAP_IN_VOLUME
0,2023-02,WEEKDAY,9,TRAIN,EW32,DT24,1,898
1,2023-02,WEEKDAY,9,TRAIN,EW32,NE17/PTC,2,898
2,2023-02,WEEKDAY,9,TRAIN,EW32,NS11,8,898
3,2023-02,WEEKDAY,9,TRAIN,EW32,EW5,1,898
4,2023-02,WEEKDAY,9,TRAIN,EW32,EW14/NS26,5,898
...,...,...,...,...,...,...,...,...
729315,2023-02,WEEKDAY,0,TRAIN,EW31,EW32,1,4
729318,2023-02,WEEKENDS/HOLIDAY,0,TRAIN,TE7,TE6,2,2
729319,2023-02,WEEKENDS/HOLIDAY,5,TRAIN,TE12,TE18,1,2
729320,2023-02,WEEKENDS/HOLIDAY,5,TRAIN,TE12,NS11,1,2


In [167]:
df1 = df1.sort_values(by=['TIME_PER_HOUR','DESTINATION_PT_CODE'])

In [168]:
total_trips = df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR", "DAY_TYPE"]).sum().reset_index().rename({"TOTAL_TRIPS":"TOTAL_TRIPS_AGG"},axis=1)

C:\Users\leish\AppData\Local\Temp\ipykernel_20792\3499177105.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_trips = df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR", "DAY_TYPE"]).sum().reset_index().rename({"TOTAL_TRIPS":"TOTAL_TRIPS_AGG"},axis=1)


In [169]:
df1 = pd.merge(df1, total_trips, on=["ORIGIN_PT_CODE","TIME_PER_HOUR", "DAY_TYPE"])

In [171]:
df1['PROB_TRIP'] = df1.apply(lambda x: round(x['TOTAL_TRIPS']/x['TOTAL_TRIPS_AGG'],3),axis=1)

In [172]:
df1['CUMSUM'] = df1.groupby(['ORIGIN_PT_CODE','TIME_PER_HOUR','DAY_TYPE'])['PROB_TRIP'].cumsum()


In [173]:
df1

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,TOTAL_TRIPS_AGG,PROB_TRIP,CUMSUM
0,2023-02,WEEKDAY,0,TRAIN,TE8,BP6/DT1,1,45,0.022,0.022
1,2023-02,WEEKDAY,0,TRAIN,TE8,CC12,1,45,0.022,0.044
2,2023-02,WEEKDAY,0,TRAIN,TE8,CC16,1,45,0.022,0.066
3,2023-02,WEEKDAY,0,TRAIN,TE8,EW24/NS1,1,45,0.022,0.088
4,2023-02,WEEKDAY,0,TRAIN,TE8,NS10,1,45,0.022,0.110
...,...,...,...,...,...,...,...,...,...,...
623351,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,EW31,EW32,1,19,0.053,0.686
623352,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,EW31,EW33,2,19,0.105,0.791
623353,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,EW31,NE4/DT19,1,19,0.053,0.844
623354,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,EW31,NS13,1,19,0.053,0.897


In [174]:
df1.to_csv("master_prob_trip.csv")

In [175]:
df1 = df1.drop(['YEAR_MONTH','PT_TYPE','PROB_TRIP','TOTAL_TRIPS_AGG','TOTAL_TRIPS'], axis=1)

In [176]:
df1.to_csv("prob_trip.csv")

In [123]:
df1.shape

(392492, 10)

In [145]:
df1 = df1.drop(df1.loc[(df1.ORIGIN_PT_CODE.str.startswith("B")) | (df1.ORIGIN_PT_CODE.str.startswith("P"))| (df1.ORIGIN_PT_CODE.str.startswith("S"))].index)

In [125]:
df1

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,TOTAL_TRIPS_AGG,PROB_TRIP,CUMSUM
0,2023-02,WEEKDAY,0,TRAIN,TE8,BP6/DT1,1,45,0.022,0.022
1,2023-02,WEEKDAY,0,TRAIN,TE8,CC12,1,45,0.022,0.044
2,2023-02,WEEKDAY,0,TRAIN,TE8,CC16,1,45,0.022,0.066
3,2023-02,WEEKDAY,0,TRAIN,TE8,EW24/NS1,1,45,0.022,0.088
4,2023-02,WEEKDAY,0,TRAIN,TE8,NS10,1,45,0.022,0.110
...,...,...,...,...,...,...,...,...,...,...
392463,2023-02,WEEKDAY,23,TRAIN,EW31,NS2,11,82,0.134,0.936
392464,2023-02,WEEKDAY,23,TRAIN,EW31,NS3,1,82,0.012,0.948
392465,2023-02,WEEKDAY,23,TRAIN,EW31,NS5,2,82,0.024,0.972
392466,2023-02,WEEKDAY,23,TRAIN,EW31,NS7,1,82,0.012,0.984


In [146]:
df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR", "DAY_TYPE"]).max().reset_index().sort_values(by="CUMSUM")

,ORIGIN_PT_CODE,TIME_PER_HOUR,DAY_TYPE,YEAR_MONTH,PT_TYPE,DESTINATION_PT_CODE,TOTAL_TRIPS,TOTAL_TRIPS_AGG,PROB_TRIP,CUMSUM
417,CC7,21,WEEKDAY,2023-02,TRAIN,TE8,169,2028,0.083,0.967
77,CC14,21,WEEKDAY,2023-02,TRAIN,TE8,235,2024,0.116,0.968
2551,TE3,20,WEEKDAY,2023-02,TRAIN,TE8,437,2011,0.217,0.968
58,CC12,22,WEEKDAY,2023-02,TRAIN,TE8,127,673,0.189,0.971
2471,TE16,20,WEEKDAY,2023-02,TRAIN,TE8,131,2023,0.065,0.971
...,...,...,...,...,...,...,...,...,...,...
2581,TE5,10,WEEKDAY,2023-02,TRAIN,TE8,193,1991,0.097,1.026
2232,NS28,20,WEEKDAY,2023-02,TRAIN,TE6,29,666,0.044,1.026
2381,TE12,10,WEEKDAY,2023-02,TRAIN,TE8,308,1983,0.155,1.028
309,CC28,13,WEEKDAY,2023-02,TRAIN,TE8,445,1970,0.226,1.028


In [147]:
df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR"]).sum().reset_index()['TOTAL_TRIPS']

C:\Users\leish\AppData\Local\Temp\ipykernel_20792\69142423.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR"]).sum().reset_index()['TOTAL_TRIPS']


0          59
1        2491
2       12686
3       20024
4       17697
        ...  
2650     6269
2651     4693
2652     5231
2653     3191
2654     1054
Name: TOTAL_TRIPS, Length: 2655, dtype: int64

In [180]:
df1

,DAY_TYPE,TIME_PER_HOUR,ORIGIN_PT_CODE,DESTINATION_PT_CODE,CUMSUM
0,WEEKDAY,0,TE8,BP6/DT1,0.022
1,WEEKDAY,0,TE8,CC12,0.044
2,WEEKDAY,0,TE8,CC16,0.066
3,WEEKDAY,0,TE8,EW24/NS1,0.088
4,WEEKDAY,0,TE8,NS10,0.110
...,...,...,...,...,...
623351,WEEKENDS/HOLIDAY,23,EW31,EW32,0.686
623352,WEEKENDS/HOLIDAY,23,EW31,EW33,0.791
623353,WEEKENDS/HOLIDAY,23,EW31,NE4/DT19,0.844
623354,WEEKENDS/HOLIDAY,23,EW31,NS13,0.897
